# Aviation Trends


There are four international airports in Syria - Damascus International Airport (DAM), Aleppo International Airport (ALP), Bassel Al-Assad International Airport (LTK) and Qamishli International Airport (KAC). DAM and ALP are bigger than the other two.This notebook analyses trends of flights in DAM, ALP and LTK airports.

## Data 

The dataset used for this update is from [Aviation Stack](https://aviationstack.com/documentation).This data is purchased through their API subscription. The data was validated in Lebanon using FlightRadar and the resulting flight details were a close match although not entirely identical. The validation exercise for Syria is still ongoing. 


In [2]:
import pandas as pd

Define a function to get a range of dates for the desirable date range

In [93]:
from datetime import datetime, timedelta


def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days) + 1):
        yield (start_date + timedelta(n)).strftime("%Y-%m-%d")


# Define the start and end dates
start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 10, 15)

In [106]:
# Define empty dataset to concat all the arrivals and departures
departures = pd.DataFrame()
arrivals = pd.DataFrame()

In [107]:
# run the API to collect arrivals and departures
import requests
import urllib3
from tqdm import tqdm

# Suppress only the InsecureRequestWarning from urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# arr_iata_values = ['LOS', 'KWI', 'SHJ']
# arr_iata_values = ','.join(arr_iata_values)

for one_day in tqdm(daterange(start_date, end_date)):
    params = {
        "access_key": "12925396bddce38222e47eed53e76e6c",
        "dep_iata": "LTK",
        #'arr_iata': 'LTK',
        "limit": "100",
        # To get the next 100 flights we need to change the offset
        # "offset": "100",
        "flight_date": one_day,
    }

    api_result = requests.get(
        "https://api.aviationstack.com/v1/flights", params, verify=False
    )

    api_response = api_result.json()

    df = pd.DataFrame(api_response["data"])

    departures = pd.concat([departures, df])

289it [14:50,  3.08s/it]


In [108]:
# Check if the number of flights per day are 100. If they are exactly 100 we need to rerun the API to get the next 100 flight.
departures["flight_date"].value_counts()

flight_date
2024-01-02    2
2024-01-03    2
2024-06-30    1
2024-07-30    1
2024-08-20    1
             ..
2024-05-05    1
2024-04-28    1
2024-04-23    1
2024-04-21    1
2024-10-15    1
Name: count, Length: 74, dtype: int64

In [104]:
# Check if the number of flights per day are 100. If they are exactly 100 we need to rerun the API to get the next 100 flight.
arrivals.sort_values(by="flight_date", ascending=False)["flight_date"].value_counts()

flight_date
2024-01-02    2
2024-01-03    2
2024-08-20    2
2024-05-05    1
2024-04-14    1
             ..
2024-06-30    1
2024-06-25    1
2024-06-23    1
2024-06-18    1
2024-06-11    1
Name: count, Length: 70, dtype: int64

In [109]:
departures.to_csv("../../data/aviation/aviationstack_ltk_01012024_15102024_dep.csv")
# arrivals.to_csv('../../data/aviation/aviationstack_ltk_01012024_14102024_arr.csv')

# arrivals.to_csv('../../data/aviation/aviationstack_bey_26082024_01012024_arr.csv')

Functions to clean the database and explode columns

In [110]:
import ast


def safe_literal_eval(value):
    if isinstance(value, str):
        try:
            return ast.literal_eval(value)
        except (ValueError, SyntaxError):
            return value
    return value


def explode(flights):
    flights["arrival"] = flights["arrival"].apply(safe_literal_eval)
    flights["departure"] = flights["departure"].apply(safe_literal_eval)

    fr1 = pd.json_normalize(flights["arrival"]).add_suffix("_arr")
    fr2 = pd.json_normalize(flights["departure"]).add_suffix("_dep")

    flights_exploded = pd.concat(
        [flights.drop(columns=["arrival", "departure"]), fr1, fr2], axis=1
    )

    return flights_exploded

In [38]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import (
    ColumnDataSource,
    DatetimeTickFormatter,
    FixedTicker,
    HoverTool,
    Label,
    Span,
    Tabs,
    TabPanel,
)
from bokeh.palettes import Spectral6


def get_area_plot(
    df, title, source_text, acled_events_daily, reindex_freq="D", events_dict={}
):
    complete_date_range = pd.date_range(
        start=df["flight_date"].min(), end=df["flight_date"].max(), freq=reindex_freq
    )
    # Pivot the data to have flight_status as columns
    df_pivot = df.pivot_table(
        index="flight_date", columns="flight_status", values="iata_arr", aggfunc="sum"
    )
    df_reindexed = df_pivot.reindex(complete_date_range).fillna(0)
    df_reindexed = df_reindexed.reset_index().rename(columns={"index": "flight_date"})

    df["total_flights"] = df.groupby("flight_date")["iata_arr"].transform("sum")
    if acled_events_daily["nrEvents"].max() < 500:
        max_flight = acled_events_daily["nrEvents"].max() + 50
    else:
        max_flight = acled_events_daily["nrEvents"].max() + 200

    # Create a ColumnDataSource
    source = ColumnDataSource(df_reindexed)

    # Create a Bokeh figure
    p = figure(
        x_axis_type="datetime",
        height=600,
        width=800,
        title=title,
        toolbar_location="above",
        tools="pan,box_zoom,reset,save",
        x_axis_label="Flight Date",
        y_axis_label="Nr Flights",
    )

    # Create a stacked area plot using varea_stack
    status_list = list(df["flight_status"].unique())
    colors = Spectral6[
        : len(status_list)
    ]  # Adjust the color palette to the number of flight statuses

    p.varea_stack(
        stackers=status_list,
        x="flight_date",
        color=colors,
        source=source,
        legend_label=status_list,
    )
    p.line(
        x=acled_events_daily["event_date"],
        y=acled_events_daily["nrEvents"],
        line_width=2,
        line_color="black",
        legend_label="Number of Conflict Events w/o Protests",
    )

    # Customize the plot
    p.y_range.start = 0
    p.y_range.end = max_flight

    # Format x-axis to show only the first day of each month
    p.xaxis.formatter = DatetimeTickFormatter(months="%b %Y", days="%d %b %Y")

    # Limit the number of ticks by using FixedTicker
    date_range = pd.date_range(
        start=df["flight_date"].min(), end=df["flight_date"].max(), freq="W"
    )
    ticks = [pd.to_datetime(date).timestamp() * 1000 for date in date_range]

    # complete_date_range = pd.date_range(start=df['flight_date'].min(), end=df['flight_date'].max())

    # Only keep a few ticks for clarity
    p.xaxis.ticker = FixedTicker(
        ticks=ticks[::2]
    )  # Adjust the slicing (e.g., [::2], [::3]) for more or fewer ticks

    # Rotate x-axis labels for better readability
    p.xaxis.major_label_orientation = 1.2

    # Enable clickable legend
    p.legend.click_policy = "mute"

    hover = p.select(dict(type=HoverTool))
    hover.tooltips = [
        ("Flight Date", "@flight_date{%F}"),
        ("Status", "$name"),
        ("Value", "@$name"),
    ]
    hover.formatters = {"@flight_date": "datetime"}

    # p.legend.title = 'Flight Status'
    p.legend.location = "top_left"
    p.legend.orientation = "horizontal"

    source_label = Label(
        x=0,
        y=0,
        x_units="screen",
        y_units="screen",
        text=source_text,
        text_font_size="10pt",
        text_color="gray",
    )

    # Add the label to the plot
    p.add_layout(source_label, "below")

    if events_dict:
        used_y_positions = []

        for index, (event_date, label) in enumerate(events_dict.items()):
            # Convert event_date to a timestamp in milliseconds (as expected by Bokeh for datetime axes)
            event_date_dt = pd.to_datetime(event_date)
            event_timestamp = event_date_dt.timestamp() * 1000

            # Create a vertical line (Span) at the event date
            span = Span(
                location=event_timestamp,
                dimension="height",
                line_color="#C6C6C6",
                line_width=2,
                line_dash=(4, 4),
            )
            p.renderers.append(span)

            # Determine a base y position for the label
            base_y = max_flight - (
                max_flight / 5
            )  # Adjust for visibility above the plot
            y_position = base_y  # Default position

            # Adjust y_position if it overlaps with previous labels
            while y_position in used_y_positions:
                y_position -= base_y / 10  # Move down until it's free

            used_y_positions.append(y_position)  # Store the used position

            # Add a label near the vertical line, aligned to the left
            event_label = Label(
                x=event_timestamp,
                y=y_position,
                x_offset=-5,  # Offset to move the label to the left of the line
                text=label,
                text_align="right",  # Align the text to the right so it stays to the left of the line
                text_color="black",
                text_font_size="10pt",
                background_fill_color="grey",
                background_fill_alpha=0.2,
            )
            p.add_layout(event_label)

    return p
    # Show the plot
    # show(p)

In [136]:
departures = pd.concat(
    [
        pd.read_csv("../../data/aviation/aviationstack_dam_01012024_14102024_dep.csv"),
        pd.read_csv("../../data/aviation/aviationstack_alp_01012024_15102024_dep.csv"),
        pd.read_csv("../../data/aviation/aviationstack_ltk_01012024_15102024_dep.csv"),
    ]
)
departures.drop(columns="Unnamed: 0", inplace=True)
departures.drop_duplicates(inplace=True)
departures.reset_index(drop=True, inplace=True)

In [137]:
departures_exploded = explode(departures)
# departures_exploded = departures_exploded[~(departures_exploded['airport_arr']==departures_exploded['airport_dep'])]

In [138]:
import pandas as pd

df = departures
# Assuming df is your DataFrame and 'flight_date' is the column with the dates
df["flight_date"] = pd.to_datetime(
    df["flight_date"]
)  # Ensure 'flight_date' is in datetime format

# Generate the complete date range from the minimum to the maximum date
complete_date_range = pd.date_range(
    start=df["flight_date"].min(), end=df["flight_date"].max()
)

# Find missing dates by comparing the complete date range with the dates in the DataFrame
missing_dates = complete_date_range.difference(df["flight_date"])

if missing_dates.empty:
    print("All dates are present.")
else:
    print(
        f"The following dates are not available in the data. The reasons are unknown.: {missing_dates}"
    )

The following dates are not available in the data. The reasons are unknown.: DatetimeIndex(['2024-01-28', '2024-01-29', '2024-01-30', '2024-04-29',
               '2024-04-30', '2024-06-07', '2024-06-08', '2024-06-09'],
              dtype='datetime64[ns]', freq=None)


## Analysing Number of Departures from Beirut International Airport in 2024

In [139]:
beginning = departures["flight_date"].min()
end = departures["flight_date"].max()
print(f"Data is available from {beginning} to {end}")

Data is available from 2024-01-01 00:00:00 to 2024-10-15 00:00:00


Conduct a duplication check for flights. If the flight is taking off from the same place, to the same place at the same time and has two entries, it is a duplicate flight

In [140]:
before = departures_exploded.shape[0]
print(f"There were {before} flights before duplication check")
# check for duplicate flights i.e., flights scheduled to take off at the exact same time from the same place to the same destination
departures_exploded = departures_exploded.drop_duplicates(
    subset=["flight_date", "scheduled_arr", "iata_arr", "iata_dep", "scheduled_dep"]
)

after = departures_exploded.shape[0]
print(
    f"There are {after} flights after duplication check. {before-after} flights were duplicated"
)

There were 4497 flights before duplication check
There are 4464 flights after duplication check. 33 flights were duplicated


### Flight Status Legend
- Scheduled: A flight that we have a schedule or flight plan for that hasn’t departed or has been canceled.
- Active: A flight that either left the gate or the runway and is on its way to its destination.
- Landed or Arrived: A flight that landed on the runway or arrived at the gate at the destination.
- Canceled: A flight that one or more data sources have indicated is canceled.
- Redirected: The flight is being redirected to another airport.
- Diverted: A flight that has landed or arrived at the gate of an airport where it wasn’t scheduled to arrive.
- Unknown: We were unable to detect the final arrival status.

In [116]:
acled_events_daily = pd.read_csv(
    "../../data/conflict/acled_daily_20240101_20241014.csv"
)
acled_events_daily["event_date"] = acled_events_daily["event_date"].apply(
    lambda x: pd.to_datetime(x)
)
acled_events_daily = acled_events_daily[acled_events_daily["event_date"] > "2024-01-01"]
acled_events_daily.drop(columns=["Unnamed: 0"], inplace=True)

In [117]:
acled_events_daily = (
    acled_events_daily.groupby([pd.Grouper(key="event_date", freq="D")])[
        ["nrFatalities", "nrEvents"]
    ]
    .sum()
    .reset_index()
)
acled_events_weekly = (
    acled_events_daily.groupby([pd.Grouper(key="event_date", freq="W")])[
        ["nrFatalities", "nrEvents"]
    ]
    .sum()
    .reset_index()
)

In [141]:
# Test to see if any flight has more than one flight status assoctaed with it.
duplicate_status_test = (
    departures_exploded.groupby(
        ["flight_date", "scheduled_arr", "iata_arr", "iata_dep", "scheduled_dep"]
    )[["flight_status"]]
    .count()
    .reset_index()
)
duplicate_status_test[duplicate_status_test["flight_status"] > 1]

,flight_date,scheduled_arr,iata_arr,iata_dep,scheduled_dep,flight_status


In [119]:
# missing_dates

events = {
    "2024-06-09": "Airline data\nnot available",
    "2024-08-25": "Israeli strikes in\nSouthern Lebanon",
    "2024-09-23": "Start of the Israel-Hezboullah\nConflict",
}

In [142]:
output_notebook()  # Display plots inline in a Jupyter notebook

complete_date_range = pd.date_range(
    start=df["flight_date"].min(), end=df["flight_date"].max()
)
df = (
    departures_exploded.groupby(["flight_date", "flight_status"])
    .count()[["iata_arr"]]
    .reset_index()
)
df["flight_date"] = df["flight_date"].apply(lambda x: pd.to_datetime(x))

show(
    get_area_plot(
        df,
        "Daily Departures from DAM, ALP and LTK",
        "Source: Flight data from AviationStack and conflict events from ACLED",
        acled_events_daily,
        events_dict=events,
    )
)

Loading BokehJS ...

### Findings
 

In [51]:
iata_mapping = {
    "BEY": "Beirut",
    "ADA": "Adana",
    "ADD": "Addis Ababa",
    "AMM": "Amman",
    "AUH": "Abu Dhabi",
    "BAH": "Bahrain",
    "BER": "Berlin",
    "BGW": "Baghdad",
    "CAI": "Cairo",
    "CDG": "Paris (Charles de Gaulle)",
    "DOH": "Doha",
    "DUS": "Düsseldorf",
    "DXB": "Dubai",
    "FCO": "Rome (Fiumicino)",
    "FRA": "Frankfurt",
    "GVA": "Geneva",
    "IST": "Istanbul",
    "JED": "Jeddah",
    "KAC": "Kuwait City",
    "KWI": "Kuwait City",
    "LCA": "Larnaca",
    "LHR": "London (Heathrow)",
    "LXR": "Luxor",
    "MAD": "Madrid",
    "NJF": "Najaf",
    "ORY": "Paris (Orly)",
    "ATH": "Athens",
    "AYT": "Antalya",
    "BRU": "Brussels",
    "BSR": "Basra",
    "CMF": "Chambéry",
    "CPH": "Copenhagen",
    "DMM": "Dammam",
    "EBL": "Erbil",
    "ESB": "Ankara",
    "IKA": "Tehran",
    "ACC": "Accra",
    "ADJ": "Amman",
    "ALP": "Aleppo",
    "BRE": "Bremen",
    "DAM": "Damascus",
    "ISU": "Sulaymaniyah",
    "AKT": "Akrotiri",
    "EVN": "Yerevan",
    "ARN": "Stockholm",
    "BZZ": "Brize Norton",
    "AZI": "Abu Dhabi (Al Bateen)",
    "ADB": "Izmir",
    "HAM": "Hamburg",
    "HKG": "Hong Kong",
    "FIH": "Kinshasa",
    "CMB": "Colombo",
    "NCE": "Nice",
    "MFM": "Macau",
    "MNL": "Manila",
    "OTP": "Bucharest",
    "RUH": "Riyadh",
    "LOS": "Lagos",
    "MLA": "Malta",
    "MRS": "Marseille",
    "PSA": "Pisa",
    "ISL": "Istanbul (Sabiha Gökçen)",
    "ALG": "Algiers",
    "SAW": "Istanbul (Sabiha Gökçen)",
    "SHJ": "Sharjah",
    "SSH": "Sharm El Sheikh",
    "DWC": "Dubai (Al Maktoum)",
    "PFO": "Paphos",
    "MCT": "Muscat",
    "CIA": "Rome (Ciampino)",
    "LBG": "Paris (Le Bourget)",
    "LEJ": "Leipzig",
    "BIA": "Bastia",
    "BUD": "Budapest",
    "CHQ": "Chania",
    "HAN": "Hanoi",
    "AMS": "Amsterdam",
    "VIY": "Vigo",
    "PNH": "Phnom Penh",
    "BRI": "Bari",
    "EIN": "Eindhoven",
    "NBO": "Nairobi",
    "CEQ": "Cannes",
    "LYS": "Lyon",
    "KTM": "Kathmandu",
    "SIR": "Sion",
    "BEG": "Belgrade",
    "INI": "Niš",
    "RMS": "Ramstein",
    "ISB": "Islamabad",
    "ZAZ": "Zaragoza",
    "HRG": "Hurghada",
    "KGS": "Kos",
    "MED": "Medina",
    "MHD": "Mashhad",
    "HER": "Heraklion",
    "ASW": "Aswan",
    "DUB": "Dublin",
    "HBE": "Alexandria",
    "ABJ": "Abidjan",
    "CTA": "Catania",
    "BOJ": "Burgas",
    "LDE": "Lourdes",
    "DEL": "Delhi",
    "LAD": "Luanda",
    "BJZ": "Badajoz",
    "MIR": "Monastir",
    "MXP": "Milan (Malpensa)",
    "HYD": "Hyderabad",
    "KRK": "Krakow",
    "BJV": "Bodrum",
    "GYD": "Baku",
    "TLV": "Tel Aviv",
    "WAW": "Warsaw",
    "DLM": "Dalaman",
    "IBZ": "Ibiza",
    "BLQ": "Bologna",
    "FAB": "Farnborough",
    "FLR": "Florence",
    "BRQ": "Brno",
    "BJY": "Belgrade (Batajnica)",
    "GOT": "Gothenburg",
    "DBB": "Dabaa",
    "CGN": "Cologne",
    "FOG": "Foggia",
    "FJR": "Fujairah",
    "CFU": "Corfu",
    "ABV": "Abuja",
    "COV": "Coventry",
    "BUS": "Batumi",
    "ZRH": "Zurich",
    "UAB": "Incirlik",
    "ULH": "Al Ula",
    "TBS": "Tbilisi",
    "STN": "London (Stansted)",
    "XJD": "Al Udeid",
    "TEB": "Teterboro",
    "SPX": "Spangdahlem",
    "VAR": "Varna",
    "LIN": "Milan (Linate)",
    "TMP": "Tampere",
    "VCE": "Venice",
    "LTN": "London (Luton)",
    "SKG": "Thessaloniki",
    "TRN": "Turin",
    "TUN": "Tunis",
    "THR": "Tehran",
    "OSR": "Ostrava",
    "SOF": "Sofia",
    "TRS": "Trieste",
    "LIS": "Lisbon",
    "JMK": "Mykonos",
    "RHO": "Rhodes",
    "KYE": "Kiryat Shmona",
    "RKE": "Roskilde",
    "RIX": "Riga",
    "TIV": "Tivat",
    "NAP": "Naples",
    "OLB": "Olbia",
    "MUH": "Marsa Matruh",
    "VIE": "Vienna",
    "SZX": "Shenzhen",
}

In [143]:
departures_exploded["airportcity"] = departures_exploded["iata_arr"].map(iata_mapping)
most_changed_departures = (
    pd.DataFrame(
        departures_exploded[
            departures_exploded["flight_status"].isin(
                ["scheduled", "cancelled", "diverted"]
            )
        ]["airportcity"]
        .value_counts()
        .head(10)
    )
    .reset_index()
    .sort_values(by="count", ascending=True)
)
most_cancelled_departures = (
    pd.DataFrame(
        departures_exploded[departures_exploded["flight_status"].isin(["cancelled"])][
            "airportcity"
        ]
        .value_counts()
        .head(10)
    )
    .reset_index()
    .sort_values(by="count", ascending=True)
)
most_scheduled_departures = (
    pd.DataFrame(
        departures_exploded[departures_exploded["flight_status"].isin(["scheduled"])][
            "airportcity"
        ]
        .value_counts()
        .head(10)
    )
    .reset_index()
    .sort_values(by="count", ascending=True)
)
most_diverted_departures = (
    pd.DataFrame(
        departures_exploded[departures_exploded["flight_status"].isin(["diverted"])][
            "airportcity"
        ]
        .value_counts()
        .head(10)
    )
    .reset_index()
    .sort_values(by="count", ascending=True)
)

In [144]:
output_notebook()
from bokeh.layouts import column
from bokeh.models import Div

# Example datasets stored in a dictionary
datasets = {
    "Most Changed Departures": most_changed_departures,
    "Most Cancelled Departures": most_cancelled_departures,
    "Most Scheduled Departures": most_scheduled_departures,
    "Most Diverted Departures": most_diverted_departures,
}

# Initialize an empty list to store the tabs
# List of colors to use for each dataset
colors = ["lightblue", "orange", "#023436", "#00BFB3"]

# Initialize an empty list to store the tabs
tabs = []

# Loop through each dataset and its corresponding color
for (title, data), color in zip(datasets.items(), colors):
    # Convert data to ColumnDataSource for Bokeh plotting
    source = ColumnDataSource(data)

    # Create a figure for each dataset
    p = figure(
        y_range=data["airportcity"],
        width=600,
        height=400,
        title=title + " in 2024",
        tools="save",
        toolbar_location="above",
    )
    p.hbar(
        y="airportcity",
        right="count",
        height=0.5,
        color=color,
        alpha=0.7,
        source=source,
    )

    # Add text inside the bars (5 points inside the bar)
    p.text(
        x="count",
        y="airportcity",
        text="count",
        source=source,
        x_offset=-5,  # This places the text 5 units inside the bar
        y_offset=-3,  # Vertical alignment
        text_align="right",
        text_baseline="middle",
        text_color="black",
        text_font_size="10pt",
    )

    # Customize plot (optional)
    p.xaxis.visible = False
    p.ygrid.visible = False
    p.xgrid.visible = False
    p.yaxis.major_tick_line_color = None  # No major ticks
    p.yaxis.minor_tick_line_color = None  # No minor ticks
    p.yaxis.axis_line_color = None
    p.outline_line_color = None

    subtitle = Div(
        text="<p style='font-size:12px; color:gray;'>Source: AviationStack. Acessed October 7th 2024.</p>",
        width=600,
    )

    layout = column(p, subtitle)

    # Create a panel with the plot and the corresponding title
    tab = TabPanel(child=layout, title=title)

    # Append the panel to the list of tabs
    tabs.append(tab)

# Arrange the tabs in a Tabs layout
tabs_layout = Tabs(tabs=tabs)

main_title = Div(
    text="<h2>Top 10 Destinations with Changes in Departures from  DAM, ALP and LTK in 2024</h2>",
    width=800,
)

# Use column layout to stack the title on top of the tabs
layout = column(main_title, tabs_layout)

# Show the tabs
show(layout)

Loading BokehJS ...

### Findings 



In [146]:
output_notebook()  # Display plots inline in a Jupyter notebook

departures_exploded["flight_date"] = departures_exploded["flight_date"].apply(
    lambda x: pd.to_datetime(x)
)
df = (
    departures_exploded.groupby(
        [pd.Grouper(key="flight_date", freq="W"), "flight_status"]
    )
    .count()[["iata_arr"]]
    .reset_index()
)

show(
    get_area_plot(
        df,
        "Weekly Departures from DAM, ALP, LTK",
        "Source: Flight data from AviationStack and conflict events from ACLED",
        acled_events_weekly,
        reindex_freq="W",
        events_dict=events,
    )
)

Loading BokehJS ...

In [147]:
arrivals = pd.concat(
    [
        pd.read_csv("../../data/aviation/aviationstack_dam_01012024_14102024_arr.csv"),
        pd.read_csv("../../data/aviation/aviationstack_alp_01012024_14102024_arr.csv"),
        pd.read_csv("../../data/aviation/aviationstack_ltk_01012024_14102024_arr.csv"),
    ]
)
arrivals.drop(columns="Unnamed: 0", inplace=True)
arrivals.drop_duplicates(inplace=True)
arrivals.reset_index(drop=True, inplace=True)

In [148]:
import pandas as pd

df = arrivals
# Assuming df is your DataFrame and 'flight_date' is the column with the dates
df["flight_date"] = pd.to_datetime(
    df["flight_date"]
)  # Ensure 'flight_date' is in datetime format

# Generate the complete date range from the minimum to the maximum date
complete_date_range = pd.date_range(
    start=df["flight_date"].min(), end=df["flight_date"].max()
)

# Find missing dates by comparing the complete date range with the dates in the DataFrame
missing_dates = complete_date_range.difference(df["flight_date"])

if missing_dates.empty:
    print("All dates are present.")
else:
    print(
        f"Flights for the following dates are missing in the data, reasons are unknown.: {missing_dates}"
    )

Flights for the following dates are missing in the data, reasons are unknown.: DatetimeIndex(['2024-01-29', '2024-01-30', '2024-04-29', '2024-04-30',
               '2024-06-07', '2024-06-08', '2024-06-09'],
              dtype='datetime64[ns]', freq=None)


In [149]:
arrivals_exploded = explode(arrivals)

In [150]:
before = arrivals_exploded.shape[0]
print(f"There were {before} flights before duplication check")
# check for duplicate flights i.e., flights scheduled to take off at the exact same time from the same place to the same destination
arrivals_exploded = arrivals_exploded.drop_duplicates(
    subset=["flight_date", "scheduled_arr", "iata_arr", "iata_dep", "scheduled_dep"]
)

after = arrivals_exploded.shape[0]
print(
    f"There are {after} flights after duplication check. {before-after} flights were duplicated"
)

There were 4415 flights before duplication check
There are 4396 flights after duplication check. 19 flights were duplicated


In [151]:
# Test to see if any flight has more than one flight status assoctaed with it.
duplicate_status_test = (
    arrivals_exploded.groupby(
        ["flight_date", "scheduled_arr", "iata_arr", "iata_dep", "scheduled_dep"]
    )[["flight_status"]]
    .count()
    .reset_index()
)
duplicate_status_test[duplicate_status_test["flight_status"] > 1]

,flight_date,scheduled_arr,iata_arr,iata_dep,scheduled_dep,flight_status


In [153]:
output_notebook()  # Display plots inline in a Jupyter notebook

df = df = (
    arrivals_exploded.groupby(["flight_date", "flight_status"])
    .count()[["iata_arr"]]
    .reset_index()
)
df["flight_date"] = df["flight_date"].apply(lambda x: pd.to_datetime(x))

show(
    get_area_plot(
        df,
        "Daily Arrivals to DAM, ALP and LTK",
        "Source: Flight data from AviationStack and conflict events from ACLED",
        acled_events_daily,
        events_dict=events,
    )
)

Loading BokehJS ...

### Findings

- Cross verifying AviationStack with FlightRadar data for 14th October shows one missing flight to Moscow and an additional flight to KWI in the early hours. Data Lab team to conduct further validation exercises to understand this discrepancy. 

In [154]:
output_notebook()  # Display plots inline in a Jupyter notebook

arrivals_exploded["flight_date"] = arrivals_exploded["flight_date"].apply(
    lambda x: pd.to_datetime(x)
)
df = (
    arrivals_exploded.groupby(
        [pd.Grouper(key="flight_date", freq="W"), "flight_status"]
    )
    .count()[["iata_arr"]]
    .reset_index()
)

show(
    get_area_plot(
        df,
        "Weekly Arrivals to DAM, ALP, LTK",
        "Source: Flight data from AviationStack and conflict events from ACLED",
        acled_events_weekly,
        reindex_freq="W",
        events_dict=events,
    )
)

Loading BokehJS ...

### Findings



In [155]:
arrivals_exploded["airportcity"] = arrivals_exploded["iata_dep"].map(iata_mapping)
most_changed_arrivals = (
    pd.DataFrame(
        arrivals_exploded[
            arrivals_exploded["flight_status"].isin(
                ["scheduled", "cancelled", "diverted"]
            )
        ]["airportcity"]
        .value_counts()
        .head(10)
    )
    .reset_index()
    .sort_values(by="count", ascending=True)
)
most_cancelled_arrivals = (
    pd.DataFrame(
        arrivals_exploded[arrivals_exploded["flight_status"].isin(["cancelled"])][
            "airportcity"
        ]
        .value_counts()
        .head(10)
    )
    .reset_index()
    .sort_values(by="count", ascending=True)
)
most_scheduled_arrivals = (
    pd.DataFrame(
        arrivals_exploded[arrivals_exploded["flight_status"].isin(["scheduled"])][
            "airportcity"
        ]
        .value_counts()
        .head(10)
    )
    .reset_index()
    .sort_values(by="count", ascending=True)
)
most_diverted_arrivals = (
    pd.DataFrame(
        arrivals_exploded[arrivals_exploded["flight_status"].isin(["diverted"])][
            "airportcity"
        ]
        .value_counts()
        .head(10)
    )
    .reset_index()
    .sort_values(by="count", ascending=True)
)

In [156]:
output_notebook()
from bokeh.layouts import column
from bokeh.models import Div

# Example datasets stored in a dictionary
datasets = {
    "Most Changed Arrivals": most_changed_arrivals,
    "Most Cancelled Arrivals": most_cancelled_arrivals,
    "Most Scheduled Arrivals": most_scheduled_arrivals,
    "Most Diverted Arrivals": most_diverted_arrivals,
}

# Initialize an empty list to store the tabs
# List of colors to use for each dataset
colors = ["lightblue", "orange", "#023436", "#00BFB3"]

# Initialize an empty list to store the tabs
tabs = []

# Loop through each dataset and its corresponding color
for (title, data), color in zip(datasets.items(), colors):
    # Convert data to ColumnDataSource for Bokeh plotting
    source = ColumnDataSource(data)

    # Create a figure for each dataset
    p = figure(
        y_range=data["airportcity"],
        width=600,
        height=400,
        title=title + " in 2024",
        tools="save",
        toolbar_location="above",
    )
    p.hbar(
        y="airportcity",
        right="count",
        height=0.5,
        color=color,
        alpha=0.7,
        source=source,
    )

    # Add text inside the bars (5 points inside the bar)
    p.text(
        x="count",
        y="airportcity",
        text="count",
        source=source,
        x_offset=-5,  # This places the text 5 units inside the bar
        y_offset=-3,  # Vertical alignment
        text_align="right",
        text_baseline="middle",
        text_color="black",
        text_font_size="10pt",
    )

    # Customize plot (optional)
    p.xaxis.visible = False
    p.ygrid.visible = False
    p.xgrid.visible = False
    p.yaxis.major_tick_line_color = None  # No major ticks
    p.yaxis.minor_tick_line_color = None  # No minor ticks
    p.yaxis.axis_line_color = None
    p.outline_line_color = None

    subtitle = Div(
        text="<p style='font-size:12px; color:gray;'>Source: AviationStack. Acessed October 7th 2024.</p>",
        width=600,
    )

    layout = column(p, subtitle)

    # Create a panel with the plot and the corresponding title
    tab = TabPanel(child=layout, title=title)

    # Append the panel to the list of tabs
    tabs.append(tab)

# Arrange the tabs in a Tabs layout
tabs_layout = Tabs(tabs=tabs)

main_title = Div(
    text="<h2>Top 10 Destinations with Changes in Arrivals to BEY in 2024</h2>",
    width=800,
)

# Use column layout to stack the title on top of the tabs
layout = column(main_title, tabs_layout)

# Show the tabs
show(layout)

Loading BokehJS ...

### Findings



In [157]:
arrivals_exploded.to_csv(
    "../../data/aviation/processed_arrivals_bey_20240101_20241014.csv"
)
departures_exploded.to_csv(
    "../../data/aviation/processed_departures_bey_20240101_20241014.csv"
)